In [1]:
import pandas as pd
import numpy as np
import os
#Mac
os.chdir('/Users/yaircardenasavila/Desktop/Github_repo')

In [8]:
cols_2026 = ['Clave Unidad Presupuestal', 'Unidad Presupuestal', 'Clave Unidad Responsable', 'Unidad Responsable', 'Clave Programa Presupuestario', 'Programa Presupuestario', 
        'Clave Componente', 'Componente', 'Clave Destino del Gasto Específico', 'Destino del Gasto Específico',
        'Clave Fuente de Financiamiento', 'Fuente de Financiamiento', 'Techo Comunicado', 'Monto Determinado', 'Adicional']
base_2026 = pd.read_excel('HH_INDEX/Fases de anteproyecto 2026 actualización de componentes transversalidad (1).xlsx', skiprows=1, usecols=cols_2026,
                          dtype={'Clave Unidad Presupuestal':str,
                                'Unidad Presupuestal':str,
                                'Clave Unidad Responsable':str,
                                'Unidad Responsable':str,
                                'Clave Programa Presupuestario':str,
                                'Programa Presupuestario':str,
                                'Clave Componente':str,
                                'Componente':str,
                                'Clave Destino del Gasto Específico':str,
                                'Destino del Gasto Específico':str,
                                'Clave Fuente de Financiamiento':str,
                                'Fuente de Financiamiento':str,
                                'Techo Comunicado':float,
                                'Monto Determinado':float,
                                'Adicional':float})

base_2026 = base_2026.rename(columns={'Concepto Capitulo 2':' NOMBRE Capitulo 2',
                              'Capitulo 2': 'CLAVE Capitulo 2'})
base_2026['llave'] = base_2026['Clave Unidad Presupuestal'] + '_' + base_2026['Clave Unidad Responsable']

In [14]:
prueba = base_2026.groupby(by = ['llave', 'Clave Programa Presupuestario',  'Clave Componente']).agg({'Monto Determinado':'sum', 'Adicional':'sum'})
prueba

Monto Determinado  \
llave  Clave Programa Presupuestario Clave Componente                      
01_000 A01                           01                        7500052.0   
                                     02                         289951.0   
02_000 A02                           04                         435155.0   
                                     05                        6582270.0   
                                     08                         108330.0   
...                                                                  ...   
57_000 SRT                           A1                        4567144.0   
                                     A2                       51535935.0   
                                     A3                         600050.0   
                                     A4                       18850068.0   
58_000 A58                           C1                      150000000.0   

                                                         Adicional  
llave  Clave Programa Presupuestario Clave Componente               
01_000 A01                           01                        0.0  
                                     02                        0.0  
02_000 A02                           04                        0.0  
                                     05                        0.0  
                                     08                        0.0  
...                                                            ...  
57_000 SRT                           A1                        0.0  
                                     A2                  7230227.0  
                                     A3                 28499950.0  
                                     A4                  6896904.0  
58_000 A58                           C1                100000000.0  

[1049 rows x 2 columns]

In [9]:
base_2026.head()

,Clave Unidad Presupuestal,Unidad Presupuestal,Clave Unidad Responsable,Unidad Responsable,Clave Programa Presupuestario,Programa Presupuestario,Clave Componente,Componente,Clave Destino del Gasto Específico,Destino del Gasto Específico,Clave Fuente de Financiamiento,Fuente de Financiamiento,Techo Comunicado,Monto Determinado,Adicional,llave
0,01,Despacho del Gobernador,000,Despacho del Gobernador,A01,Atención Integral del Despacho del C. Gobernador,01,Gastos básicos para la operación,00,(Sin destino),15,Recursos Federales,4127904.0,4127904.0,0.0,01_000
1,01,Despacho del Gobernador,000,Despacho del Gobernador,A01,Atención Integral del Despacho del C. Gobernador,01,Gastos básicos para la operación,00,(Sin destino),15,Recursos Federales,26933.0,26933.0,0.0,01_000
2,01,Despacho del Gobernador,000,Despacho del Gobernador,A01,Atención Integral del Despacho del C. Gobernador,02,Atención Integral del Despacho del C. Gobernador,00,(Sin destino),15,Recursos Federales,57337.0,57337.0,0.0,01_000
3,01,Despacho del Gobernador,000,Despacho del Gobernador,A01,Atención Integral del Despacho del C. Gobernador,01,Gastos básicos para la operación,00,(Sin destino),15,Recursos Federales,573326.0,573326.0,0.0,01_000
4,01,Despacho del Gobernador,000,Despacho del Gobernador,A01,Atención Integral del Despacho del C. Gobernador,01,Gastos básicos para la operación,00,(Sin destino),15,Recursos Federales,161883.0,161883.0,0.0,01_000
